![image source: https://www.artbreeder.com/image/6b4df6c697078f0e2cda42348ec6](images/2025-02-10-diffusion-model-mnist-part2.jpeg)

## Introduction

Welcome back to Part 2 of our journey into diffusion models! In the [first part](https://hassaanbinaslam.github.io/myblog/posts/2025-02-10-diffusion-model-mnist-part1.html), we successfully built a basic Convolutional UNet from scratch and trained it to directly predict denoised MNIST digits.  We saw that it could indeed remove some noise, but the results were still a bit blurry, and it wasn't quite the "diffusion model magic" we were hoping for.

One of the key limitations we hinted at was the simplicity of our `BasicUNet` architecture.  For this second part, we're going to address that head-on. We'll be upgrading our UNet architecture to something more powerful and feature-rich.

To do this, we'll be leveraging the fantastic `diffusers` library from Hugging Face.  `diffusers` is a widely adopted toolkit in the world of diffusion models, providing pre-built and optimized components that can significantly simplify our development process and boost performance.

In this part, we'll replace our `BasicUNet` with a `UNet2DModel` from `diffusers`.  We'll keep the core task the same – direct image prediction – but with a more advanced UNet under the hood. This will allow us to see firsthand how architectural improvements can impact the quality of our denoising results, setting the stage for even more exciting explorations in future parts! Let's dive in!

### Credits

This post is inspired by the [Hugging Face Diffusion Course](https://huggingface.co/learn/diffusion-course/en/unit1/3)

### Environment Details

You can access and run this Jupyter Notebook from the GitHub repository on this link [2025-02-15-diffusion-model-mnist-part2.ipynb](https://github.com/hassaanbinaslam/myblog/blob/main/posts/2025-02-15-diffusion-model-mnist-part2.ipynb)

Run the following cell to install the required packages.

* This notebook can be run with [Google Colab](https://colab.research.google.com/) T4 GPU runtime.
* I have also tested this notebook with AWS SageMaker Jupyter Notebook running on instance "ml.g5.xlarge" and image "SageMaker Distribution 2.3.0".

In [1]:
%%capture
!pip install datasets[vision]
!pip install diffusers
!pip install watermark
!pip install torchinfo
!pip install matplotlib

[WaterMark](https://github.com/rasbt/watermark) is an IPython magic extension for printing date and time stamps, version numbers, and hardware information. Let's load this extension and print the environment details.

In [2]:
%load_ext watermark

In [3]:
%watermark -v -m -p torch,torchvision,datasets,diffusers,matplotlib,watermark,torchinfo

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

torch      : 2.5.1+cu124
torchvision: 0.20.1+cu124
datasets   : 3.2.0
diffusers  : 0.32.2
matplotlib : 3.10.0
watermark  : 2.5.0
torchinfo  : 1.8.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



## Diving into `diffusers` and `UNet2DModel`

So, what exactly *is* this `diffusers` library we're so excited about?  Think of `diffusers` as a comprehensive, community-driven library in PyTorch specifically designed for working with diffusion models. It's maintained by Hugging Face, the same team behind the popular Transformers library, so you know it's built with quality and ease of use in mind.

Why are we using `diffusers` now?  Several reasons!  First, it provides well-tested and optimized implementations of various diffusion model components, saving us from writing everything from scratch.  Second, it's a vibrant ecosystem, constantly evolving with the latest research and techniques in diffusion models.  By using `diffusers`, we're standing on the shoulders of giants!

For Part 2, the star of the show is the `UNet2DModel` class from `diffusers`. This is a more sophisticated UNet architecture compared to our `BasicUNet`.  It's like upgrading from a standard bicycle to a mountain bike – both are bikes, but the mountain bike is built for more challenging terrain and better performance.

What makes `UNet2DModel` more advanced? Let's look at some key architectural improvements under the hood:

*   **ResNet Blocks:**  Instead of simple convolutional layers, `UNet2DModel` utilizes ResNet blocks within its downsampling and upsampling paths. ResNet blocks are known for making it easier to train deeper networks, which can capture more complex features in images. Think of them as more efficient and powerful building blocks for our UNet.

*   **Attention Mechanisms:**  `UNet2DModel` incorporates attention mechanisms, specifically "Attention Blocks," in its architecture.  Attention is a powerful concept in deep learning that allows the model to focus on the most relevant parts of the input when processing information.  In image generation, attention can help the model selectively focus on different regions of the image, potentially leading to finer details and more coherent structures.

*   **Group Normalization:**  Instead of Batch Normalization, `UNet2DModel` uses Group Normalization. Group Normalization is often favored in generative models, especially when working with smaller batch sizes, as it tends to be more stable and perform better in those scenarios.

*   **Timestep Embedding:**  Even though we are still doing direct image prediction in this part,  `UNet2DModel` is designed with diffusion models in mind.  It includes a `TimestepEmbedding` layer, which is a standard component in diffusion models to handle the timestep information (which we'll explore in later parts!).  For now, we'll just be passing in a timestep of 0, but this layer is there, ready for when we move to true diffusion.

These architectural enhancements in `UNet2DModel` give it a greater capacity to learn and potentially denoise images more effectively than our `BasicUNet`. Let's see if it lives up to the hype!